In [1]:
#initial setup

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import random
import PIL
import torch
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torchvision import transforms, utils, models
from torch import nn
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
from mlxtend.plotting import plot_confusion_matrix

In [2]:
#defining data directory and listing subfolders with flower images

data_dir = '../input/flowers-recognition/flowers/'
folders = os.listdir(data_dir)
print(folders)

In [3]:
#Dataframe with image paths and corresponding classes
df = pd.DataFrame(columns=["path","class"])

for i in folders:
  path = os.path.join(data_dir, i)
  for j in os.listdir(path):
    if os.path.splitext(j)[1]==".jpg":
        df=df.append({"path":os.path.join(path, j),"class": i},ignore_index=True)

In [4]:
# Print summary of a DataFrame to get data type, columns, non-null values and memory usage

df.info()

In [5]:
# count number of images per class 

df["class"].value_counts()

In [7]:
# bar chart showing the distribution of images by classes

plt.bar(df["class"].value_counts().index, height=df["class"].value_counts().values)
plt.show()

In [8]:
#Modified code from https://www.kaggle.com/sachinsharma1123/flowers-fun
# same as previous one but represented in pie chart

plt.figure(figsize=(18,10))
plt.pie(df["class"].value_counts().values,labels=df["class"].value_counts().index,startangle=90,colors=['r','g','b','y','m'],autopct='%1.1f%%',explode = (0, 0.1, 0, 0,0),shadow=True)
plt.legend()
plt.show()

In [9]:
#Visualization of the data

plt.figure(figsize=(10, 10))
for i in range(9):
    index=random.choice(df.index)
    ax = plt.subplot(3, 3, i + 1)
    img = np.asarray(PIL.Image.open(df["path"][index]))
    plt.imshow(img)
    plt.title(df["class"][index])
    plt.axis("off")

In [10]:
#Finding all the unique formats and dimensions of images in the dataset

dimensions=[]
channels=[]

for row in df.iterrows():
    img = PIL.Image.open(row[1]["path"])

    dimensions.append(np.asarray(img).shape[0:2])
    channels.append(np.asarray(img).shape[2])
    
print("Dimensions of the pictures in the dataset are:", set(dimensions))
print("\n\nChannels of the pictures in the dataset are:", set(channels))

In [11]:
#Class label encoding to transform non-numerical labels (images) to numerical labels

label_enc = LabelEncoder()
df['class'] = label_enc.fit_transform(df['class'])

In [12]:
# check if the transformation was successful

df.head()

In [13]:
#Train - validation - test split :  70-20-10 %, stratified by class
#https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn

train_ratio = 0.70
validation_ratio = 0.20
test_ratio = 0.10

# train is now 70% of the entire data set
train,  test = train_test_split(df, test_size=1 - train_ratio, random_state = 0, stratify=df["class"])

# test is now 20% of the initial data set
# validation is now 10% of the initial data set
validation, test = train_test_split(test, test_size=test_ratio/(test_ratio + validation_ratio), random_state = 0, stratify=test["class"]) 


print("Train set shape:", train.shape)
print("Validation set shape:", validation.shape)
print("Test set shape:", test.shape)

In [14]:
#Distribution of classes in train set
plt.bar(label_enc.inverse_transform(train["class"].value_counts().index), height=train["class"].value_counts().values)
plt.show()

In [15]:
#Distribution of classes in validation set
plt.bar(label_enc.inverse_transform(validation["class"].value_counts().index), height=validation["class"].value_counts().values)
plt.show()

In [16]:
#Distribution of classes in test set
plt.bar(label_enc.inverse_transform(test["class"].value_counts().index), height=test["class"].value_counts().values)
plt.show()

In [17]:
# Defining dataloaders

dataset =torch.utils.data.Dataset

class Flowers(dataset):
    def __init__(self, dataframe,transform=None):
        self.dataframe = dataframe
        self.transform = transform
        
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        image=PIL.Image.open(row["path"])
        
        if self.transform:
            image = self.transform(image)
            
        return (image,row["class"])

In [18]:
#To overcome the problem of overfitting the train images were transformed
#Normalization means and standard deviations for all imagenet models

transform_train = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ColorJitter(brightness=0.15, contrast=0.2, saturation=0.1),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])])
    
transform_test = transforms.Compose([transforms.Resize(224),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

In [19]:
#Preparation of data for trainig with data loaders

train_dataset = Flowers(train,transform=transform_train)
valid_dataset = Flowers(validation,transform=transform_test)
test_dataset = Flowers(test,transform=transform_test)

In [20]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=72, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=72)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=72)

In [21]:
#Code from homework
dataiter = iter(train_dataloader)
images = dataiter.next()
plt.imshow(utils.make_grid(images[0][0:8],nrow=4).permute(1,2,0),aspect='auto')

In [22]:
#Device definition
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [23]:
#Downloading pretrained googlenet model
model=models.googlenet(pretrained=True)

In [24]:
#Checking how the models looks, and number of inputs of last layer
model

In [25]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
        param.requires_grad = False

In [26]:
#Code from HW - changing last layer
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 512)), # Layer before last one gives 1024 outputs
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(512, 5)), # 5 outputs - 5 classes of flowers
                          ]))
    
model.fc = classifier

In [27]:
#Just to check if it was added properly
model

In [28]:
#Defining Criterion  and optimizer and transferring model to device
criterion = nn.CrossEntropyLoss() # Combines log softmax and NLL loss
model.to(device)
optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.001)

In [29]:
#Training code from homework - modified
epochs = 100
running_loss = 0

train_losses, valid_losses = [], []
for epoch in range(epochs):
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        


    valid_loss = 0
    accuracy = 0
    with torch.no_grad():
        model.eval()
        for inputs, labels in valid_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)   
            valid_loss += batch_loss.item()       
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
    train_losses.append(running_loss/len(train_dataloader))
    valid_losses.append(valid_loss/len(valid_dataloader))
    print(f"Epoch {epoch+1}/{epochs}.. "
          f"Train loss: {train_losses[-1]:.3f}.. "
          f"Validation loss: {valid_losses[-1]:.3f}.. "
          f"Validation accuracy: {accuracy/len(valid_dataloader):.3f}")
    running_loss = 0
    model.train()

In [30]:
#Visualizing losses

epoch_list = list(range(1,epochs+1))

plt.figure()
plt.plot(epoch_list,train_losses, label='Train Loss')
plt.plot(epoch_list,valid_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc = 'best')
plt.show()

In [31]:
#Modified helper function from https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/
def evaluate_model(test_dataloader, model):
    predictions, actuals = list(), list()
    model.eval()
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(test_dataloader):
            # evaluate the model on the test set
            yhat = model(inputs)
            # retrieve numpy array
            yhat = yhat.detach().numpy()
            actual = targets.numpy()
            # convert to class labels
            yhat = np.argmax(yhat, axis=1)
            # reshape for stacking
            actual = actual.reshape((len(actual), 1))
            yhat = yhat.reshape((len(yhat), 1))
            # store
            predictions.append(yhat)
            actuals.append(actual)
    predictions, actuals = np.vstack(predictions), np.vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc, predictions, actuals

In [32]:
#Pushing model back to cpu for predicitions
model.to("cpu"); # ; to supress output of the cell, we already know how the model looks

In [33]:
# Evaluating the model accuracy on test set
acc, predictions, actuals = evaluate_model(test_dataloader, model)
print('Test Accuracy: %.3f' % acc)

In [34]:
print(classification_report(actuals, predictions, target_names=label_enc.classes_))

In [35]:
# Plot confusion matrix - https://www.kaggle.com/georgiisirotenko/pytorch-flowers-translearing-ensemble-test-99-67

cm  = confusion_matrix(actuals, predictions)
plt.figure()
plot_confusion_matrix(cm,figsize=(12,8),cmap=plt.cm.Blues)
plt.xticks(range(len(label_enc.classes_)), label_enc.classes_, fontsize=16)
plt.yticks(range(len(label_enc.classes_)), label_enc.classes_, fontsize=16)
plt.xlabel('Predicted Label',fontsize=18)
plt.ylabel('True Label',fontsize=18)
plt.show()